In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
from io import StringIO
import requests
import json
import plotly.express as px
import ipywidgets as widgets
from datetime import datetime


Populating the interactive namespace from numpy and matplotlib


# Get Covid data

In [2]:
URL_province_covid="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province-"
start_date = datetime(2020, 2, 24)
end_date = datetime(2020, 3, 31)
possible_dates = pd.date_range(start=start_date, end=end_date, freq='D')


def getLatestFiles(url, possible_dates):
    files={}
    for d in possible_dates:
        datestring= d.strftime("%Y%m%d")
        s=requests.get(url + datestring +".csv")
        if s.status_code == 200:
            df = pd.read_csv(StringIO(s.text))
            df = df.dropna(subset=['sigla_provincia'])
            df[['totale_casi']] = df[['totale_casi']].astype(int)
            files[d] = df
            print("got file for date %s" % datestring)
            # print(df.shape)
        else: 
            print("no file for date %s" % datestring)
    return files

files = getLatestFiles(URL_province_covid, possible_dates)
last_file_date = max(k for k, v in files.items())
print("last available file for %s " % last_file_date)
files[last_file_date].head()



got file for date 20200224
got file for date 20200225
got file for date 20200226
got file for date 20200227
got file for date 20200228
got file for date 20200229
got file for date 20200301
got file for date 20200302
got file for date 20200303
got file for date 20200304
got file for date 20200305
got file for date 20200306
got file for date 20200307
got file for date 20200308
got file for date 20200309
got file for date 20200310
got file for date 20200311
got file for date 20200312
got file for date 20200313
got file for date 20200314
got file for date 20200315
got file for date 20200316
got file for date 20200317
got file for date 20200318
got file for date 20200319
got file for date 20200320
got file for date 20200321
got file for date 20200322
no file for date 20200323
no file for date 20200324
no file for date 20200325
no file for date 20200326
no file for date 20200327
no file for date 20200328
no file for date 20200329
no file for date 20200330
no file for date 20200331
last avail

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi
0,2020-03-22 17:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,102
1,2020-03-22 17:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,38
2,2020-03-22 17:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,334
3,2020-03-22 17:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,113
5,2020-03-22 17:00:00,ITA,17,Basilicata,77,Matera,MT,40.667512,16.597924,16


# Get GeoJson for Italy

In [3]:
URL_province_geojson="https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson"

def getGeoJson(url):
    try: 
        s=requests.get(url)
        return json.load(StringIO(s.text))
    except HTTPError as e:
        # Need to check its an 404, 503, 500, 403 etc.
        print("http error %s" % e.response.status_code)

geojson_province = getGeoJson(URL_province_geojson)

len([g['properties']['prov_acr'] for g in geojson_province['features']])
    

107

# Maps

In [ ]:
def province_map(file, title):
    fig = px.choropleth(file, geojson=geojson_province, 
                        color="totale_casi",
                        locations="sigla_provincia", 
                        featureidkey="properties.prov_acr",
                        projection="mercator",
                        hover_data=["denominazione_provincia", "sigla_provincia", "totale_casi"]
                       )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        title_text = title,
        title_font_size=20
        #geo_scope='europe', # limite map scope to Europe
    )
    
    return fig

In [ ]:
available_dates = pd.date_range(start=start_date, end=last_file_date, freq='D')
options = [(date.strftime(' %d/%m/%Y '), date) for date in available_dates]

selection_slider = widgets.SelectionSlider(
    options = options,
    description='Date',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '600px'}
)

In [85]:
def update_map(date):
    title = "Total cases for %s "% date.strftime("%d %b %Y")
    fig = province_map(files[date], title )
    fig.show()
    

widgets.interact(
    update_map,
    date=selection_slider
);

interactive(children=(SelectionSlider(continuous_update=False, description='Date', layout=Layout(width='600px'…

In [82]:
def diff_dates(date_range):
    title = "Total increase between %s and %s " % (date_range[0].strftime("%d %b %Y"), date_range[1].strftime("%d %b %Y"))
    diff = files[date_range[0]].loc[:, ('sigla_provincia','denominazione_provincia' ,'totale_casi')]
    diff = diff.rename(columns={"totale_casi": "totale_casi_first"})
    diff['totale_casi_second']= files[date_range[1]].loc[:, ('totale_casi')]
    diff['totale_casi'] = diff['totale_casi_second'] - diff['totale_casi_first']
    diff.head()
    fig = province_map(diff, title)
    fig.show()
    print("done")
    


In [88]:
index = (0, len(options)-1)
selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '600px'}
)


In [89]:

widgets.interact(
    diff_dates,
    date_range=selection_range_slider
);


interactive(children=(SelectionRangeSlider(continuous_update=False, description='Dates', index=(0, 27), layout…

In [205]:

widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')